In [29]:
import os

import meshio
import numpy as np
from fimpy.solver import FIMPY
from nutils import export, function, mesh
from nutils.expression_v2 import Namespace

from sklearn.neighbors import BallTree
from numba import njit, prange
from numba.typed import List


In [30]:
# Read with nutils
mesh_name = os.path.join("control_arm", "control_arm.msh")
domain, geom = mesh.gmsh(mesh_name)

# Read with meshio
msh = meshio.read(mesh_name)
points = msh.points
elem = msh.cells[0].data

# Injection location
injection_locs = [[-25.000, 111.916, 18.961]]

gmsh > loaded 3d gmsh topology consisting of #694049 elements



In [31]:
# Prepare nutils variables
ns = Namespace()
ns.x = geom
ns.define_for("x", gradient="∇", normal="n", jacobians=("dV", "dS"))
ns.basis = domain.basis("std", degree=1)
ns.D = function.dotarg("lhs", ns.basis)

In [32]:
# Compute distance to surface walls
D = np.stack([np.eye(3)] * len(elem))
faces = np.vstack(
    [elem[:, [0, 1, 2]], elem[:, [1, 2, 3]], elem[:, [0, 2, 3]], elem[:, [0, 1, 3]]]
)
faces = np.sort(faces, axis=1)
_, indices, counts = np.unique(faces, axis=0, return_counts=True, return_index=True)
indices = indices[np.argwhere(counts == 1)]
surface_nodes = np.unique(faces[indices]).ravel()

In [33]:
fim = FIMPY.create_fim_solver(points, elem, D, device="cpu", use_active_list=False)
D_w = fim.comp_fim(surface_nodes, np.zeros_like(surface_nodes))

In [34]:
# Compute weighted distance to inlet
inlet_nodes = []
for injection_loc in injection_locs:
    dist_to_inlet = np.linalg.norm(points - injection_loc, axis=-1)
    inlet_nodes.append(np.argmin(dist_to_inlet, axis=0))
inlet_nodes = np.array(inlet_nodes)
D_w_elem = np.mean(D_w[elem], axis=1)
D2 = np.einsum("i,jk->ijk", D_w_elem, np.eye(3))

In [35]:
fim2 = FIMPY.create_fim_solver(points, elem, D2, device="cpu")
D_i2 = fim2.comp_fim(inlet_nodes, np.zeros_like(inlet_nodes))

In [36]:
@njit(parallel=True)
def compute_structure_tensor(points, centers, volumes, indices, dist):
    num_points = len(points)
    result = np.zeros((num_points, 3, 3))
    for i in prange(num_points):
        idx = indices[i]
        diffs = centers[idx] - points[i]
        vols = volumes[idx]
        dirs = diffs.T / dist[i]
        result[i, :, :] = np.dot(vols * dirs, dirs.T) / np.sum(vols)
    return result


def tensor_sweep(points, centers, volumes, radius):
    tree = BallTree(centers)
    results = []
    chunks = np.array_split(points, 100)
    for chunk in chunks:
        n, d = tree.query_radius(chunk, r=radius, return_distance=True)
        S = compute_structure_tensor(chunk, centers, volumes, List(n), List(d))
        results.append(S)
    return np.concatenate(results)

In [37]:
bounding_box = np.max(points, axis=0) - np.min(points, axis=0)
cell_coords = np.concatenate([points[block.data] for block in msh.cells])
centers = np.mean(cell_coords, axis=1)

# Get node to cell relations
node2cells = {k: [] for k in range(len(points))}
for block in msh.cells:
    for i, cell in enumerate(block.data):
        for j, node in enumerate(cell):
            node2cells[node].append(i)

# Get cell volumes
a = cell_coords[:, 1, :] - cell_coords[:, 0, :]
b = cell_coords[:, 2, :] - cell_coords[:, 0, :]
c = cell_coords[:, 3, :] - cell_coords[:, 0, :]
omega = np.einsum("ij,ij->i", a, np.cross(b, c), optimize=True)
cell_volumes = np.abs(omega) / 6.0

# Interpolate volume field to points
node_volumes = np.array(
    [np.sum(cell_volumes[node2cells[node]] / 4.0) for node in range(len(points))]
)
msh.point_data["Volume"] = node_volumes

In [38]:
# Compute structure tensor
radius = min(0.25 * np.max(bounding_box), np.min(bounding_box))
stensors = tensor_sweep(points, centers, cell_volumes, radius)
msh.point_data["Structure_tensor"] = stensors.reshape((-1, 9))[:, [0, 4, 8, 1, 5, 2]]

In [39]:
# Evaluate on mesh
bezier = domain.sample("vtk", 2)
D_w = bezier.eval("D" @ ns, lhs=D_w)

In [40]:
x, D_i2, dD_i2 = bezier.eval(["x_i", "D", "∇_i(D)"] @ ns, lhs=D_i2)

In [41]:
# Export result
export.vtk(mesh_name[:-4], bezier.tri, x, D_w=D_w, D_i2=D_i2, dD_i2=dD_i2)

control_arm/control_arm.vtk


In [42]:
# Reorder mesh
indices = np.lexsort(np.fliplr(msh.points).T)
reverse_indices = np.argsort(indices)
points = msh.points[indices]
cells = []
for cell_block in msh.cells:
    cell_block.data = reverse_indices[cell_block.data]
    cells.append(cell_block)
point_data = {}
for label, pdata in msh.point_data.items():
    point_data[label] = pdata[indices]

nutils_mesh = meshio.vtk.read(mesh_name.replace(".msh", ".vtk"))
_, indices = np.unique(nutils_mesh.points, return_index=True, axis=0)
for label, pdata in nutils_mesh.point_data.items():
    point_data[label] = pdata[indices]

result = meshio.Mesh(points=points, cells=cells, point_data=point_data)
meshio.write("control_arm_hole.vtu", result)